## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import config from another folder
import sys
sys.path.append("../")

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Venice,US,27.0998,-82.4543,84.22,73,0,3.44,haze
1,1,Punta Arenas,CL,-53.1500,-70.9167,46.51,61,0,13.80,clear sky
2,2,Dikson,RU,73.5069,80.5464,36.32,77,99,8.12,overcast clouds
3,3,Tahe,CN,52.3333,124.7333,49.39,76,98,4.41,overcast clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,41.00,81,100,10.36,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature (F) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (F) you would like for your trip? "))

What is the minimum temperature (F) you would like for your trip? 75
What is the maximum temperature (F) you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Venice,US,27.0998,-82.4543,84.22,73,0,3.44,haze
8,8,Saint George,US,37.1041,-113.5841,75.79,57,0,9.22,clear sky
11,11,Praia,CV,14.9215,-23.5087,80.02,81,6,3.49,clear sky
16,16,Lorengau,PG,-2.0226,147.2712,81.64,80,50,9.28,scattered clouds
17,17,Bata,GQ,1.8639,9.7658,75.58,87,95,4.88,light rain
20,20,Kavaratti,IN,10.5669,72.6420,80.29,81,100,20.92,overcast clouds
29,29,Butaritari,KI,3.0707,172.7902,81.18,73,3,11.30,clear sky
30,30,Hambantota,LK,6.1241,81.1185,77.88,83,96,16.13,overcast clouds
32,32,Kapaa,US,22.0752,-159.3190,82.72,73,40,8.05,scattered clouds
38,38,Anamur,TR,36.0751,32.8369,82.33,43,0,8.66,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                236
City                   236
Country                236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Current Description    236
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.NaN
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Venice,US,84.22,haze,27.0998,-82.4543,NaN
8,Saint George,US,75.79,clear sky,37.1041,-113.5841,NaN
11,Praia,CV,80.02,clear sky,14.9215,-23.5087,NaN
16,Lorengau,PG,81.64,scattered clouds,-2.0226,147.2712,NaN
17,Bata,GQ,75.58,light rain,1.8639,9.7658,NaN
20,Kavaratti,IN,80.29,overcast clouds,10.5669,72.6420,NaN
29,Butaritari,KI,81.18,clear sky,3.0707,172.7902,NaN
30,Hambantota,LK,77.88,overcast clouds,6.1241,81.1185,NaN
32,Kapaa,US,82.72,scattered clouds,22.0752,-159.3190,NaN
38,Anamur,TR,82.33,clear sky,36.0751,32.8369,NaN


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Venice,US,84.22,haze,27.0998,-82.4543,A Beach Retreat On Casey Key
8,Saint George,US,75.79,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
11,Praia,CV,80.02,clear sky,14.9215,-23.5087,LT Aparthotel
16,Lorengau,PG,81.64,scattered clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
17,Bata,GQ,75.58,light rain,1.8639,9.7658,Hotel Elik Melen
20,Kavaratti,IN,80.29,overcast clouds,10.5669,72.6420,Green leaf
29,Butaritari,KI,81.18,clear sky,3.0707,172.7902,Isles Sunset Lodge
30,Hambantota,LK,77.88,overcast clouds,6.1241,81.1185,Bungalow 63
32,Kapaa,US,82.72,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
38,Anamur,TR,82.33,clear sky,36.0751,32.8369,Luna Piena Beach Hotel


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
        <dl>
        <dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
        <dt><b>City</b></dt><dd>{City}</dd>
        <dt><b>Country</b></dt><dd>{Country}</dd>
        <dt><b>Weather description</b></dt><dd>{Current Description} and max {Max Temp} °F</dd>
        </dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# Add a heatmap of temperature for the vacation spots.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
                                 dissipating=False,
             max_intensity=300, point_radius=4)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))